
# シーケンスレベルおよびトークンレベルのアプリケーション向けの BERT の微調整

:label: `sec_finetuning-bert`

この章の前のセクションでは、RNN、CNN、attention、MLP などに基づく、自然言語処理アプリケーション用のさまざまなモデルを設計しました。これらのモデルは、スペースや時間に制約がある場合に役立ちますが、すべての自然言語処理タスクに対して特定のモデルを作成することは事実上不可能です。 :numref: `sec_bert`では、幅広い自然言語処理タスクに対して最小限のアーキテクチャ変更を必要とする事前トレーニング モデル BERT を導入しました。一方では、BERT は提案時点で、さまざまな自然言語処理タスクに関する最先端の技術を向上させました。一方、 :numref: `sec_bert-pretraining`に記載されているように、元の BERT モデルの 2 つのバージョンには 1 億 1,000 万個と 3 億 4,000 万個のパラメータが付属しています。したがって、十分な計算リソースがある場合は、下流の自然言語処理アプリケーション向けに BERT を微調整することを検討することがあります。

以下では、自然言語処理アプリケーションのサブセットをシーケンス レベルおよびトークン レベルとして一般化します。シーケンス レベルでは、単一テキスト分類およびテキスト ペア分類または回帰において、テキスト入力の BERT 表現を出力ラベルに変換する方法を紹介します。トークン レベルでは、テキストのタグ付けや質問応答などの新しいアプリケーションを簡単に紹介し、BERT がどのように入力を表現し、出力ラベルに変換できるかを明らかにします。微調整中に、さまざまなアプリケーションにわたる BERT に必要な「最小限のアーキテクチャ変更」は、完全に接続された追加のレイヤーです。ダウンストリーム アプリケーションの教師あり学習中に、事前トレーニングされた BERT モデルのすべてのパラメーターが微調整される一方で、追加レイヤーのパラメーターが最初から学習されます。

## 単一テキストの分類

*単一テキスト分類は、*単一のテキスト シーケンスを入力として受け取り、その分類結果を出力します。この章で研究した感情分析に加えて、言語受容性コーパス (CoLA) は、特定の文が文法的に許容できるかどうかを判断する、単一テキスト分類のためのデータセットでもあります :cite: `Warstadt.Singh.Bowman.2019` 。たとえば、「勉強しなければなりません。」は受け入れられますが、「勉強する必要があります。」ではありません。 

![](../img/bert-one-seq.svg) :label: `fig_bert-one-seq`

 :numref: `sec_bert` BERT の入力表現を記述します。 BERT 入力シーケンスは、単一のテキストとテキストのペアの両方を明確に表し、特別な分類トークン「&amp;lt;cls&amp;gt;」が使用されます。シーケンスの分類と特別な分類トークン「&amp;lt;sep&amp;gt;」に使用されます。単一のテキストの終わりをマークするか、テキストのペアを区切ります。 :numref: `fig_bert-one-seq`に示すように、単一テキスト分類アプリケーションでは、特別な分類トークン「&amp;lt;cls&amp;gt;」の BERT 表現が使用されます。入力テキストシーケンス全体の情報をエンコードします。入力単一テキストの表現として、完全に接続された (密な) レイヤーで構成される小さな MLP に入力され、すべての離散ラベル値の分布が出力されます。

## テキストペアの分類または回帰

この章では、自然言語推論についても検討しました。これは、*テキストのペア*を分類するアプリケーションの一種であるテキスト ペア分類に属します。

テキストのペアを入力として受け取り、連続値を出力する、*意味論的なテキストの類似性は*、一般的な*テキスト ペア回帰*タスクです。このタスクは、文の意味上の類似性を測定します。たとえば、セマンティック テキスト類似性ベンチマーク データセットでは、一対の文の類似性スコアは 0 (意味の重複なし) から 5 (意味の同等性) までの順序スケールです (引用: `Cer.Diab.Agirre.ea.2017` )。目標は、これらのスコアを予測することです。セマンティック テキスト類似性ベンチマーク データセットの例には、次のものが含まれます (文 1、文 2、類似性スコア)。
-  「飛行機が離陸しています。」、「飛行機が離陸しています。」、5.000;
-  「女性が何かを食べています。」、「女性が肉を食べています。」、3.000;
-  「女性が踊っている。」「男性が話している。」、0.000。 

![](../img/bert-two-seqs.svg) :label: `fig_bert-two-seqs`

 :numref: `fig_bert-one-seq`の単一テキスト分類と比較すると、 :numref: `fig_bert-two-seqs`のテキスト ペア分類に対する BERT の微調整は、入力表現が異なります。意味的なテキストの類似性などのテキスト ペア回帰タスクでは、連続ラベル値の出力や平均二乗損失の使用などの簡単な変更を適用できます。これらは回帰では一般的です。

## テキストのタグ付け

次に、各トークンにラベルが割り当てられる、*テキストのタグ付け*などのトークンレベルのタスクを考えてみましょう。テキストタグ付けタスクのうち、*品詞タグ付けは、*文内での単語の役割に応じて各単語に品詞タグ（形容詞や限定詞など）を割り当てます。たとえば、Penn Treebank II タグ セットによれば、「ジョン スミスの車は新しいです」という文は、「NNP (名詞、固有単数) NNP POS (所有語尾) NN (名詞、単数または質量) VB」としてタグ付けされる必要があります。 （動詞、原形）JJ（形容詞）」。 

![](../img/bert-tagging.svg) :label: `fig_bert-tagging`

テキストタグ付けアプリケーション向けの BERT の微調整については、 :numref: `fig_bert-tagging`で説明します。 :numref: `fig_bert-one-seq`と比較すると、唯一の違いは、テキストのタグ付けでは、入力テキストの*すべてのトークン*の BERT 表現が同じ追加の完全に接続された層に供給されて、トークンのラベルが出力されることです。品詞タグ。

## 質問への回答

別のトークンレベルのアプリケーションとして、*質問応答は*読解力を反映します。たとえば、Stanford Question Answering Dataset (SQuAD v1.1) は、文章と質問を読むことで構成されており、すべての質問に対する答えは、質問に関する文章のテキストの一部 (テキスト スパン) です。:cite: `Rajpurkar.Zhang.Lopyrev.ea.2016` 。これを説明するには、「一部の専門家は、マスクの有効性は決定的ではないと報告している。しかし、マスクメーカーは、N95マスクなどの自社製品はウイルスを防ぐことができると主張している。」という一節を考えてみましょう。そして「N95マスクがウイルスを防ぐことができると誰が言いますか？」という質問。答えは、この文章の「マスクメーカー」というテキスト範囲であるはずです。したがって、SQuAD v1.1 の目標は、質問とパッセージのペアが与えられたパッセージ内のテキスト スパンの開始と終了を予測することです。 

![](../img/bert-qa.svg) :ラベル: `fig_bert-qa`

質問応答用に BERT を微調整するために、質問と文章がそれぞれ最初と 2 番目のテキスト シーケンスとして BERT の入力にパックされます。テキスト スパンの開始位置を予測するために、同じ追加の全結合層が、位置 $i$ の通過からの任意のトークンの BERT 表現をスカラー スコア $s_i$ に変換します。すべてのパッセージ トークンのこのようなスコアは、ソフトマックス操作によって確率分布にさらに変換されるため、パッセージ内の各トークン位置 $i$ には、テキスト スパンの開始である確率 $p_i$ が割り当てられます。テキスト スパンの終了の予測は、追加の全結合層のパラメーターが開始を予測するためのパラメーターから独立していることを除いて、上記と同じです。終了を予測するとき、位置 $i$ の通過トークンは同じ全結合層によってスカラー スコア $e_i$ に変換されます。 :numref: `fig_bert-qa`質問応答用の BERT の微調整を示しています。

質問応答の場合、教師あり学習のトレーニング目標は、グラウンドトゥルースの開始位置と終了位置の対数尤度を最大化することと同じくらい簡単です。スパンを予測する場合、位置 $i$ から位置 $j$ までの有効なスパンのスコア $s_i + e_j$ ($i \leq j$) を計算し、最も高いスコアを持つスパンを出力できます。

## まとめ
- BERT では、単一テキストの分類 (感情分析や言語的受容性のテストなど)、テキスト ペアの分類または回帰 (自然言語など) などのシーケンス レベルおよびトークン レベルの自然言語処理アプリケーションに対して最小限のアーキテクチャ変更 (追加の完全接続層) が必要です。推論および意味論的なテキストの類似性）、テキストのタグ付け（品詞タグ付けなど）、および質問応答。
- ダウンストリーム アプリケーションの教師あり学習中に、事前トレーニングされた BERT モデルのすべてのパラメーターが微調整される一方で、追加レイヤーのパラメーターが最初から学習されます。

## 演習
1. ニュース記事の検索エンジン アルゴリズムを設計してみましょう。システムがクエリ (例: 「コロナウイルス流行中の石油業界」) を受信すると、そのクエリに最も関連性の高いニュース記事のランク付けされたリストを返す必要があります。膨大な量のニュース記事と多数のクエリがあるとします。問題を単純化するために、クエリごとに最も関連性の高い記事にラベルが付けられていると仮定します。アルゴリズム設計でネガティブ サンプリング (:numref: `subsec_negative-sampling`参照) と BERT を適用するにはどうすればよいでしょうか?
1. 言語モデルのトレーニングで BERT をどのように活用できるでしょうか?
1. 機械翻訳で BERT を活用できますか?

[ディスカッション](https://discuss.d2l.ai/t/396)
